# Unit 3

## Creating Your Own Signature in DSPy

## Your First Sentiment Classifier Signature

## Creating Your First Summarization Signature

## Building a Multi-Input Question Answering Signature

## Creating Emotion Classifiers with Class Signatures

## Building a Fact Checking Signature

## Building a Fact Checking Signature